In [1]:
import pandas as pd

In [2]:
mypath = "/home/mmarzec12/data/"

df = pd.read_csv(mypath+"bgg_ratings_merged_filtered_2015_10_500.csv")

explicit = df[df.score!=0]
implicit = df[df.score==0]

In [23]:
def customf(df):
    max_score_id = df.score.values.argmax()
    max_game_id = df.game_id.iloc[max_score_id]
    
    d = {"max_game_id":max_game_id,
        "score":df.score.iloc[max_score_id]}
        
    return pd.Series(d) #max_game_id

# create test set
df_test = explicit.groupby("user_name").apply(customf)
#df_test["score"] = df_test.apply(lambda x: )
#print(df_test.head())
df_test = df_test.reset_index().rename({"max_game_id":"game_id"}, axis=1)
df_test["game_id"] = df_test["game_id"].astype(int)

#print(df_test.head())
# create explicit train set (df_explicit - df_test)
df_train = pd.concat([df_test, explicit[["user_name", "game_id", "score"]]]).drop_duplicates(keep=False)

#print(df_train.head())
# create validation set
df_validation = df_train.groupby("user_name").apply(customf)
df_validation = df_validation.reset_index().rename({"max_game_id":"game_id"}, axis=1)
df_validation["game_id"] = df_validation["game_id"].astype(int)

# recreate sexplicit train set (df_train - df_validation)
df_train = pd.concat([df_validation, df_train]).drop_duplicates(keep=False)

In [28]:
len(df_train) + len(df_test) + len(df_validation) == len(explicit)

True

In [29]:
# saving dataframes

df_train.to_csv(mypath+"explicit_train.csv", index=False)
implicit.to_csv(mypath+"implicit_train.csv", index=False)
df_test.to_csv(mypath+"leave_one_out_test.csv", index=False)
df_test.to_csv(mypath+"leave_one_out_validation.csv", index=False)